# *1. Instalar paquetes necesarios*

In [1]:
''' Instale paquetes de MySQL y de lectura de archivos
    de Excel, y de geoanalitica'''

!pip install mysql-connector-python openpyxl geopandas folium # Son estos los paquetes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.9/33.9 MB 15.8 MB/s eta 0:00:00


# *2. Importar posibles librerías necesarias*

In [2]:
''' Importar aquí las librerías necesarias'''

import mysql.connector          #Para conectarse a SQL
import pandas as pd             #Para manipular los datos
import os                       #Interacción con la máquina
import glob                     #Encontrar ruta
import re                       #Encontrar patrones de texto
import matplotlib.pyplot as plt #Hacer Gráficos
import numpy as np              #Hacer operaciones con los datos
import geopandas as gpd         #Hacer Geo analitica

# *3. Conceder permiso a Google drive para acceder a los archivos subidos*

In [3]:
''' Conceder permiso a Google drive para acceder a los archivos subidos'''
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# *4. Leer las rutas de los archivos en Google Drive*

In [4]:
''' Estas son la rutas donde estan los 3 grupos de archivos'''

ruta_admitidos = "/content/drive/MyDrive/Curso Talento Tech/Proyecto Bootcamp/Bases de Datos/Admitidos"
ruta_matriculados = "/content/drive/MyDrive/Curso Talento Tech/Proyecto Bootcamp/Bases de Datos/Matriculados 1er Curso"
ruta_graduados = "/content/drive/MyDrive/Curso Talento Tech/Proyecto Bootcamp/Bases de Datos/Graduados"
ruta_drivers = "/content/drive/MyDrive/Curso Talento Tech/Proyecto Bootcamp/Bases de Datos/Drivers/Columnas Modelo.xlsx"

# *5. Leer los archivos en cada carpeta y concaternalos para crear 3 dataframes distintos*

In [5]:
''' Tomar del archivo ruta_drivers (que es un .xlsx) cada hoja por individual
    y crear un dataframe por cada hoja'''

# Driver 1
df_columnas_finales = pd.read_excel(ruta_drivers, sheet_name="Columnas_Que_Sobreviven")
df_columnas_finales = df_columnas_finales[df_columnas_finales["Cuales sobreviven"] == "x"]
# Creo la lista de las columnas que quedan finalmente
columnas_que_se_quedan = df_columnas_finales["Nombres Ajustados"].tolist()

# Driver 2
df_nombres_ajustados = pd.read_excel(ruta_drivers, sheet_name="Nombres_Columnas")
# Creo un diccionario que ajusta los nombres
diccionario_nombres = df_nombres_ajustados.set_index("Posibles Nombres Columnas")["Nombres Ajustados"].to_dict()

In [6]:
''' Crear una función que lea los archivos xlsx
    en una ruta de drive y cree un dataframe por cada archivo
    y ajuste los nombres de las columnas '''

# Esta función tiene como objetivo leer los archivos en una ruta y crear
# un dataframe por cada archivo

def concat_archivos(ruta):
  archivos = glob.glob(os.path.join(ruta, "*.xlsx"))
  lista_df = []
  for archivo in archivos:
    df = pd.read_excel(archivo)
    df = df.rename(columns=diccionario_nombres)
    lista_df.append(df)
  df_concat = pd.concat(lista_df)
  return df_concat

In [7]:
''' Crear los 3 Dataframes en función de las rutas asignadas'''

df_admitidos = concat_archivos(ruta_admitidos)
df_matriculados = concat_archivos(ruta_matriculados)
df_graduados = concat_archivos(ruta_graduados)

In [8]:
''' Ajustar bases de graduados, admitidos y matriculados'''

#Base graduados
columnas_existentes1 = [col for col in columnas_que_se_quedan if col in df_graduados.columns]
df_graduados = df_graduados[columnas_existentes1]

#Base admitidos
columnas_existentes2 = [col for col in columnas_que_se_quedan if col in df_admitidos.columns]
df_admitidos = df_admitidos[columnas_existentes2]

#Base matriculados
columnas_existentes3 = [col for col in columnas_que_se_quedan if col in df_matriculados.columns]
df_matriculados = df_matriculados[columnas_existentes3]

#Crear un dataframe final
df_Total = pd.concat([df_graduados, df_admitidos, df_matriculados])

# *6. Filtrar y validar datos del dataframe final que se piensa usar*

In [9]:
# Filtar con el campo ID CINE Amplio el número 6

df_Total = df_Total[df_Total['ID CINE CAMPO AMPLIO'] == 6]

In [10]:
# Ver estructura del dataframe
df_Total.head()

,IES PADRE,INSTITUCIÓN DE EDUCACIÓN SUPERIOR (IES),ID SECTOR IES,SECTOR IES,ID CARÁCTER IES,CARÁCTER IES,ID NIVEL ACADÉMICO,NIVEL ACADÉMICO,ID NIVEL DE FORMACIÓN,NIVEL DE FORMACIÓN,...,DESC CINE CAMPO AMPLIO,CÓDIGO DEL DEPARTAMENTO (PROGRAMA),DEPARTAMENTO DE OFERTA DEL PROGRAMA,ID SEXO,SEXO,AÑO,SEMESTRE,GRADUADOS,ADMITIDOS,PRIMER CURSO
448,1101.0,UNIVERSIDAD NACIONAL DE COLOMBIA,1.0,OFICIAL,4.0,Universidad,2.0,POSGRADO,2.0,Maestría,...,Tecnologías de la Información y la Comunicació...,11.0,"Bogotá, D.C.",1.0,Hombre,2021.0,2.0,7.0,NaN,NaN
449,1101.0,UNIVERSIDAD NACIONAL DE COLOMBIA,1.0,OFICIAL,4.0,Universidad,2.0,POSGRADO,2.0,Maestría,...,Tecnologías de la Información y la Comunicació...,11.0,"Bogotá, D.C.",2.0,Mujer,2021.0,2.0,1.0,NaN,NaN
495,1101.0,UNIVERSIDAD NACIONAL DE COLOMBIA,1.0,OFICIAL,4.0,Universidad,2.0,POSGRADO,3.0,Doctorado,...,Tecnologías de la Información y la Comunicació...,11.0,"Bogotá, D.C.",1.0,Hombre,2021.0,1.0,1.0,NaN,NaN
496,1101.0,UNIVERSIDAD NACIONAL DE COLOMBIA,1.0,OFICIAL,4.0,Universidad,2.0,POSGRADO,3.0,Doctorado,...,Tecnologías de la Información y la Comunicació...,11.0,"Bogotá, D.C.",1.0,Hombre,2021.0,2.0,2.0,NaN,NaN
575,1101.0,UNIVERSIDAD NACIONAL DE COLOMBIA,1.0,OFICIAL,4.0,Universidad,2.0,POSGRADO,1.0,Especialización universitaria,...,Tecnologías de la Información y la Comunicació...,11.0,"Bogotá, D.C.",1.0,Hombre,2021.0,1.0,3.0,NaN,NaN


In [11]:
# Validar información en columnas
df_Total.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44895 entries, 448 to 44246
Data columns (total 23 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   IES PADRE                                44895 non-null  float64
 1   INSTITUCIÓN DE EDUCACIÓN SUPERIOR (IES)  44895 non-null  object 
 2   ID SECTOR IES                            44895 non-null  float64
 3   SECTOR IES                               44895 non-null  object 
 4   ID CARÁCTER IES                          44895 non-null  float64
 5   CARÁCTER IES                             44895 non-null  object 
 6   ID NIVEL ACADÉMICO                       44895 non-null  float64
 7   NIVEL ACADÉMICO                          44895 non-null  object 
 8   ID NIVEL DE FORMACIÓN                    44895 non-null  float64
 9   NIVEL DE FORMACIÓN                       44895 non-null  object 
 10  ID MODALIDAD                             44895 no

In [12]:
# Validar valores de columnas categorícas que decidimos dejar en el modelo

print(df_Total['SECTOR IES'].unique()) #Listo
print(df_Total['CARÁCTER IES'].unique()) #Listo
print(df_Total['DEPARTAMENTO DE OFERTA DEL PROGRAMA'].unique()) #Listo
print(df_Total['NIVEL ACADÉMICO'].unique())
print(df_Total['NIVEL DE FORMACIÓN'].unique())
print(df_Total['MODALIDAD'].unique())
print(df_Total['DESC CINE CAMPO AMPLIO'].unique())
print(df_Total['SEXO'].unique())

['OFICIAL' 'PRIVADA' 'Oficial' 'Privado' 'Privada']
['Universidad' 'Institución Universitaria/Escuela Tecnológica'
 'Institución Tecnológica' 'Institución Técnica Profesional'
 'INSTITUCIÓN UNIVERSITARIA/ESCUELA TECNOLÓGICA' 'UNIVERSIDAD'
 'INSTITUCIÓN TECNOLÓGICA' 'INSTITUCIÓN TÉCNICA PROFESIONAL'
 'INSTITUCION UNIVERSITARIA/ESCUELA TECNOLOGICA' 'INSTITUCION TECNOLOGICA'
 'INSTITUCION TECNICA PROFESIONAL']
['Bogotá, D.C.' 'Antioquia' 'Caldas' 'Boyacá' 'Cauca' 'Risaralda' 'Nariño'
 'Huila' 'Caquetá' 'Meta' 'Cesar' 'Valle Del Cauca' 'Santander' 'Bolívar'
 'Córdoba' 'Tolima' 'Quindío' 'Norte De Santander' 'Magdalena'
 'Cundinamarca' 'La Guajira' 'Atlántico' 'Casanare' 'Sucre' 'Chocó'
 'Putumayo' 'Archipiélago De San Andrés, Providencia Y Santa Catalina'
 'Arauca' 'Vichada' 'Guaviare' 'Guainía' 'Bogotá D.C.' 'Valle del Cauca'
 'Norte de Santander'
 'Archipiélago de San Andrés, Providencia y Santa Catalina' 'Amazonas'
 'ATLÁNTICO' 'CAQUETÁ' 'SANTANDER' 'RISARALDA' 'ANTIOQUIA' 'BOGOTÁ D.C.'

# *7. Crear diccionarios que homologuen los nombres de las columnas categoricas*

In [13]:
''' Crear una función que permite operar texto de forma optima para poder
    homologar los datos de las columnas categoricas '''

#Libreria util para operar texto, ya se tiene Re y Pandas Arriba
import unicodedata

def limpiar_formatear_nompropio(texto):
    if pd.isnull(texto):
        return texto

    # Eliminar acentos
    texto = unicodedata.normalize("NFKD", texto).encode("ASCII", "ignore").decode("utf-8")

    # Eliminar caracteres especiales (deja solo letras y espacios)
    texto = re.sub(r"[^a-zA-Z\s]", "", texto)

    # Pasar a minúsculas
    texto = texto.lower()

    # Eliminar espacios múltiples
    texto = re.sub(r"\s+", " ", texto).strip()

    # Palabras que deben ir en minúscula si no son la primera
    palabras_minusculas = {"de", "la", "del", "y", "en", "el", "los", "las", "un", "una"}

    palabras = texto.split()
    if not palabras:
        return ""

    resultado = [palabras[0].capitalize()] + [
        p if p in palabras_minusculas else p.capitalize() for p in palabras[1:]
    ]

    return " ".join(resultado)

In [14]:
#Aplicar la función Anterior para Homologar en gran medida los datos
df_Total['SECTOR IES'] = df_Total['SECTOR IES'].apply(limpiar_formatear_nompropio) #ok
df_Total['CARÁCTER IES'] = df_Total['CARÁCTER IES'].apply(limpiar_formatear_nompropio) #ok
df_Total['DEPARTAMENTO DE OFERTA DEL PROGRAMA'] = df_Total['DEPARTAMENTO DE OFERTA DEL PROGRAMA'].apply(limpiar_formatear_nompropio) #ok
df_Total['NIVEL ACADÉMICO'] = df_Total['NIVEL ACADÉMICO'].apply(limpiar_formatear_nompropio) #ok
df_Total['NIVEL DE FORMACIÓN'] = df_Total['NIVEL DE FORMACIÓN'].apply(limpiar_formatear_nompropio) #ok
df_Total['MODALIDAD'] = df_Total['MODALIDAD'].apply(limpiar_formatear_nompropio) #ok
df_Total['DESC CINE CAMPO AMPLIO'] = df_Total['DESC CINE CAMPO AMPLIO'].apply(limpiar_formatear_nompropio) #Ok
df_Total['SEXO'] = df_Total['SEXO'].apply(limpiar_formatear_nompropio) #ok

In [15]:
# Resolver Particularidades Sector IES
df_Total['SECTOR IES'] = df_Total['SECTOR IES'].replace({
    'Oficial':'Oficial',
    'Privada':'Privada',
    'Privado':'Privada'
    })

In [16]:
# Resolver particularidades Departamento
df_Total['DEPARTAMENTO DE OFERTA DEL PROGRAMA'] = df_Total['DEPARTAMENTO DE OFERTA DEL PROGRAMA'].replace({
    'Bogota Dc':'Bogota Dc',
    'Antioquia':'Antioquia',
    'Caldas':'Caldas',
    'Boyaca':'Boyaca',
    'Cauca':'Cauca',
    'Risaralda':'Risaralda',
    'Narino':'Narino',
    'Huila':'Huila',
    'Caqueta':'Caqueta',
    'Meta':'Meta',
    'Cesar':'Cesar',
    'Valle del Cauca':'Valle del Cauca',
    'Santander':'Santander',
    'Bolivar':'Bolivar',
    'Cordoba':'Cordoba',
    'Tolima':'Tolima',
    'Quindio':'Quindio',
    'Norte de Santander':'Norte de Santander',
    'Magdalena':'Magdalena',
    'Cundinamarca':'Cundinamarca',
    'La Guajira':'La Guajira',
    'Atlantico':'Atlantico',
    'Casanare':'Casanare',
    'Sucre':'Sucre',
    'Choco':'Choco',
    'Putumayo':'Putumayo',
    'Archipielago de San Andres Providencia y Santa Catalina':'San Andres Providencia y Santa Catalina',
    'Arauca':'Arauca',
    'Vichada':'Vichada',
    'Guaviare':'Guaviare',
    'Guainia':'Guainia',
    'Amazonas':'Amazonas',
    'Guainia':'Guainia',
    'Amazonas' :'Amazonas',
    'Archipielago de Sa':'San Andres Providencia y Santa Catalina',
    'Vaupes':'Vaupes',
    'San Andres y Providencia':'San Andres Providencia y Santa Catalina',
    'Narinio':'Narino',
    'Guajira':'La Guajira',
    'San Andres y Provi':'San Andres y Provi'
    })

In [17]:
# Resolver particularidades Nivel de formación
df_Total["NIVEL DE FORMACIÓN"] = df_Total["NIVEL DE FORMACIÓN"].apply(
    lambda x: "Universitario" if x == "Universitaria"
    else "Tecnologico" if x == "Tecnologica"
    else "Especializacion Universitaria" if x == "Especializacion"
    else x
)

In [25]:
# Resolver particularidades modalidad
df_Total["MODALIDAD"] = df_Total["MODALIDAD"].apply(
    lambda x: "Hibrido" if x == "Presencialvirtual"
    else "Hibrido" if x == "Presencialdual"
    else "Hibrido" if x == "Presencialvirtuala Distancia"
    else "A Distancia" if x == "Distancia Tradicional"
    else "Virtual" if x == "Distancia Virtual"
    else "A Distancia" if x == "Distancia Tradicion"
    else "A Distancia" if x == "A Distancia Tradicional"
    else "Virtual" if x == "A Distancia Virtual"
    else x)

In [28]:
# Resolver particularidades Sexo
df_Total["SEXO"] = df_Total["SEXO"].apply(
    lambda x: "Femenino" if x == "Mujer"
    else "Masculino" if x == "Hombre"
    else "No Informa" if x == "Sin Informacion"
    else x)

In [30]:
# Exportar df_Total como .csv separados por ","
df_Total.to_csv('df_Total.csv', index=False, sep=',')